In [1]:
import pandas as pd
import numpy as np

### Convert xml to xsv

In [2]:
df = pd.read_xml('data/export.xml')
df.to_csv('data/export.csv', index=False)

### Read and transform the data

In [3]:
data = pd.read_csv("data/export.csv")
data

/var/folders/j6/hnb1wkw115x3893s7581k7s40000gn/T/ipykernel_29721/2140728767.py:1: DtypeWarning: Columns (0,1,2,3,4,5,8,9,13,15,17,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/export.csv")


,value,HKCharacteristicTypeIdentifierDateOfBirth,HKCharacteristicTypeIdentifierBiologicalSex,HKCharacteristicTypeIdentifierBloodType,HKCharacteristicTypeIdentifierFitzpatrickSkinType,HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse,type,sourceName,sourceVersion,unit,...,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,appleMoveTime,appleMoveTimeGoal,appleExerciseTime,appleExerciseTimeGoal,appleStandHours,appleStandHoursGoal,HeartRateVariabilityMetadataList
0,2025-05-31 19:35:31 +0200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2003-04-02,HKBiologicalSexMale,HKBloodTypeNotSet,HKFitzpatrickSkinTypeNotSet,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,174,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeight,iPhone de Eliandro Melo,11.0.1,cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,180,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeight,Health,12.3.1,cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,181,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeight,Health,13.1.3,cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377981,53.4268,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Eliandro’s Apple Watch,11.0,ms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377982,28.667,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Eliandro’s Apple Watch,11.0,ms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377983,30.8445,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Eliandro’s Apple Watch,11.0,ms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377984,32.1236,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Eliandro’s Apple Watch,11.0,ms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.columns

Index(['value', 'HKCharacteristicTypeIdentifierDateOfBirth',
       'HKCharacteristicTypeIdentifierBiologicalSex',
       'HKCharacteristicTypeIdentifierBloodType',
       'HKCharacteristicTypeIdentifierFitzpatrickSkinType',
       'HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse', 'type',
       'sourceName', 'sourceVersion', 'unit', 'creationDate', 'startDate',
       'endDate', 'device', 'MetadataEntry', 'workoutActivityType', 'duration',
       'durationUnit', 'WorkoutStatistics', 'WorkoutEvent', 'WorkoutRoute',
       'dateComponents', 'activeEnergyBurned', 'activeEnergyBurnedGoal',
       'activeEnergyBurnedUnit', 'appleMoveTime', 'appleMoveTimeGoal',
       'appleExerciseTime', 'appleExerciseTimeGoal', 'appleStandHours',
       'appleStandHoursGoal', 'HeartRateVariabilityMetadataList'],
      dtype='object')

#### Timestamp columns

In [5]:
data[['startDate', 'endDate', 'creationDate']]

,startDate,endDate,creationDate
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,2017-09-28 13:33:53 +0200,2017-09-28 13:33:53 +0200,2017-09-28 13:33:53 +0200
3,2019-07-12 00:14:03 +0200,2019-07-12 00:14:03 +0200,2019-07-12 00:14:03 +0200
4,2019-11-12 01:10:57 +0200,2019-11-12 01:10:57 +0200,2019-11-12 01:10:57 +0200
...,...,...,...
1377981,2025-05-28 18:10:33 +0200,2025-05-28 18:11:28 +0200,2025-05-28 18:11:34 +0200
1377982,2025-05-28 18:11:38 +0200,2025-05-28 18:12:37 +0200,2025-05-28 18:12:38 +0200
1377983,2025-05-28 19:56:52 +0200,2025-05-28 19:57:43 +0200,2025-05-28 19:57:53 +0200
1377984,2025-05-28 23:59:17 +0200,2025-05-29 00:00:17 +0200,2025-05-29 00:00:18 +0200


##### We drop every row without a value in the 'value' and 'type' column, because we won't be able to use them.

In [6]:
data = data.dropna(subset=['value', 'type'])
data

,value,HKCharacteristicTypeIdentifierDateOfBirth,HKCharacteristicTypeIdentifierBiologicalSex,HKCharacteristicTypeIdentifierBloodType,HKCharacteristicTypeIdentifierFitzpatrickSkinType,HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse,type,sourceName,sourceVersion,unit,...,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,appleMoveTime,appleMoveTimeGoal,appleExerciseTime,appleExerciseTimeGoal,appleStandHours,appleStandHoursGoal,HeartRateVariabilityMetadataList
2,174,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeight,iPhone de Eliandro Melo,11.0.1,cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,180,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeight,Health,12.3.1,cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,181,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeight,Health,13.1.3,cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,181,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeight,Eliandro’s Apple Watch,10.0,cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,177,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeight,Health,17.6.1,cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377981,53.4268,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Eliandro’s Apple Watch,11.0,ms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377982,28.667,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Eliandro’s Apple Watch,11.0,ms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377983,30.8445,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Eliandro’s Apple Watch,11.0,ms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377984,32.1236,NaN,NaN,NaN,NaN,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Eliandro’s Apple Watch,11.0,ms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### We can see here all the available 'type' values that can be used as columns

In [7]:
data.type.unique()

array(['HKQuantityTypeIdentifierHeight',
       'HKQuantityTypeIdentifierBodyMass',
       'HKQuantityTypeIdentifierHeartRate',
       'HKQuantityTypeIdentifierOxygenSaturation',
       'HKQuantityTypeIdentifierRespiratoryRate',
       'HKQuantityTypeIdentifierStepCount',
       'HKQuantityTypeIdentifierDistanceWalkingRunning',
       'HKQuantityTypeIdentifierBasalEnergyBurned',
       'HKQuantityTypeIdentifierActiveEnergyBurned',
       'HKQuantityTypeIdentifierFlightsClimbed',
       'HKQuantityTypeIdentifierAppleExerciseTime',
       'HKQuantityTypeIdentifierRestingHeartRate',
       'HKQuantityTypeIdentifierVO2Max',
       'HKQuantityTypeIdentifierWalkingHeartRateAverage',
       'HKQuantityTypeIdentifierEnvironmentalAudioExposure',
       'HKQuantityTypeIdentifierHeadphoneAudioExposure',
       'HKQuantityTypeIdentifierWalkingDoubleSupportPercentage',
       'HKQuantityTypeIdentifierSixMinuteWalkTestDistance',
       'HKQuantityTypeIdentifierAppleStandTime',
       'HKQuantityType

##### Now, we analyze the missing values on each column

In [8]:
data.isna().sum()

value                                                              0
HKCharacteristicTypeIdentifierDateOfBirth                    1376949
HKCharacteristicTypeIdentifierBiologicalSex                  1376949
HKCharacteristicTypeIdentifierBloodType                      1376949
HKCharacteristicTypeIdentifierFitzpatrickSkinType            1376949
HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse    1376949
type                                                               0
sourceName                                                         0
sourceVersion                                                  12898
unit                                                           18505
creationDate                                                       0
startDate                                                          0
endDate                                                            0
device                                                        113551
MetadataEntry                     

##### Here, we have the percentages of each column.

In [9]:
data.isna().sum() / len(df)

value                                                        0.000000
HKCharacteristicTypeIdentifierDateOfBirth                    0.999247
HKCharacteristicTypeIdentifierBiologicalSex                  0.999247
HKCharacteristicTypeIdentifierBloodType                      0.999247
HKCharacteristicTypeIdentifierFitzpatrickSkinType            0.999247
HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse    0.999247
type                                                         0.000000
sourceName                                                   0.000000
sourceVersion                                                0.009360
unit                                                         0.013429
creationDate                                                 0.000000
startDate                                                    0.000000
endDate                                                      0.000000
device                                                       0.082404
MetadataEntry       

#### Let's drop the unusable rows and columns.

In [11]:
data = data[['startDate', 'endDate', 'type', 'value', 'unit', 'creationDate']]
data

,startDate,endDate,type,value,unit,creationDate
2,2017-09-28 13:33:53 +0200,2017-09-28 13:33:53 +0200,HKQuantityTypeIdentifierHeight,174,cm,2017-09-28 13:33:53 +0200
3,2019-07-12 00:14:03 +0200,2019-07-12 00:14:03 +0200,HKQuantityTypeIdentifierHeight,180,cm,2019-07-12 00:14:03 +0200
4,2019-11-12 01:10:57 +0200,2019-11-12 01:10:57 +0200,HKQuantityTypeIdentifierHeight,181,cm,2019-11-12 01:10:57 +0200
5,2023-10-08 02:11:12 +0200,2023-10-08 02:11:12 +0200,HKQuantityTypeIdentifierHeight,181,cm,2023-10-08 02:11:12 +0200
6,2024-09-05 03:43:34 +0200,2024-09-05 03:43:34 +0200,HKQuantityTypeIdentifierHeight,177,cm,2024-09-05 03:43:34 +0200
...,...,...,...,...,...,...
1377981,2025-05-28 18:10:33 +0200,2025-05-28 18:11:28 +0200,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,53.4268,ms,2025-05-28 18:11:34 +0200
1377982,2025-05-28 18:11:38 +0200,2025-05-28 18:12:37 +0200,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,28.667,ms,2025-05-28 18:12:38 +0200
1377983,2025-05-28 19:56:52 +0200,2025-05-28 19:57:43 +0200,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,30.8445,ms,2025-05-28 19:57:53 +0200
1377984,2025-05-28 23:59:17 +0200,2025-05-29 00:00:17 +0200,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,32.1236,ms,2025-05-29 00:00:18 +0200


##### Let's check the missing values again.

In [12]:
data.isna().sum() / len(df)

startDate       0.000000
endDate         0.000000
type            0.000000
value           0.000000
unit            0.013429
creationDate    0.000000
dtype: float64

##### It seems that we still have missing values in the 'unit' column.
##### As we can see below, this only happens when the 'value' column has categorical values.

In [13]:
data[data.unit.isna()].value.unique()

array(['HKCategoryValueSleepAnalysisInBed',
       'HKCategoryValueSleepAnalysisAsleepCore',
       'HKCategoryValueSleepAnalysisAsleepDeep',
       'HKCategoryValueSleepAnalysisAwake',
       'HKCategoryValueSleepAnalysisAsleepREM',
       'HKCategoryValueSleepAnalysisAsleepUnspecified',
       'HKCategoryValueAppleStandHourStood',
       'HKCategoryValueAppleStandHourIdle',
       'HKCategoryValueNotApplicable',
       'HKCategoryValueEnvironmentalAudioExposureEventMomentaryLimit'],
      dtype=object)

##### Unique sleep analysis values

In [17]:
data[data.type == 'HKCategoryTypeIdentifierSleepAnalysis'].value.value_counts()

HKCategoryValueSleepAnalysisInBed                8472
HKCategoryValueSleepAnalysisAsleepCore            472
HKCategoryValueSleepAnalysisAsleepREM             184
HKCategoryValueSleepAnalysisAwake                 178
HKCategoryValueSleepAnalysisAsleepDeep            157
HKCategoryValueSleepAnalysisAsleepUnspecified       2
Name: value, dtype: int64